<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li></ul></div>

<h3>Overview

This is part of a series of notebooks used to impute missing location data and create ***lat*** (latitude) and ***lon*** (longitude) columns. The ***location*** column of the original dataset is a dictionary with geocordinates and business address. Many of these location entries are null. Where not null, the coordinates are pulled from the location column. Where no coordinates are present, the a geocoding api is used to convert the address fields are to lat and lon. 

Note book 1.1c **Extract Lat Lon From Address** performs the conversion by address fields.

In [1]:
import pandas as pd
import numpy as np
import json
import ast


import os
import sys
import re

module_path = os.path.abspath(os.path.join('./lib/'))

if module_path not in sys.path:   
    sys.path.append(module_path)

import datetime
from utilities import *
from api_keys import *

import matplotlib.pyplot as plt
import seaborn as sns



import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point

import os
import sys
import re
import time

%matplotlib inline

In [2]:
df_by_addr = pd.read_csv('../tmp/by_location_2019201951813674374.csv', low_memory=False)

In [3]:
df_by_addr.shape

(143883, 27)

In [4]:
df_by_addr['lat'] = 0
df_by_addr['lon'] = 0

In [5]:
df_by_addr.location.isnull().sum()

0

In [6]:
def convert_address(addr):
    time.sleep(3)
    try:
        geo = geocode(addr, provider='google', api_key=google_key)
    except:
        #print(f'an error occurred')
        return 0,0
    else:
        point = geo.geometry.geometry
        lon = float(point.x)
        lat = float(point.y)
        #print(f'returning {lat}, {lon}')
        return lat, lon

NOTE:  The following code calls the geocoding API. This was run iteratively until no more addresses could be translated. This was a manual process, and consumed a lot of resources, and for this reason I am including the code in a commented out state, to avoid have to re-run it.  

```

start_execution = time.time()

df_by_addr[['lat', 'lon']] = df_by_addr.apply(lambda row: [0, 0] if row.lat == 0 else
            test(f'{row.full_business_address}, San Francisco, CA'),
                                axis=1, result_type='expand'')

end_execution = time.time()
print(f'Elapsed time: {end_execution - start_execution}')

df_by_addr.head()

ts = get_timestamp()
df_by_addr.to_csv(f'../tmp/converted_by_addr_{ts}.csv', index=False)

```